# intro to Smartcore

Let's start installing the library dependency.

In [6]:
// cargo install the newest version
:dep smartcore = { git = "https://github.com/smartcorelib/smartcore", branch = "v0.5-wip" }

## numbers types

As an example of what Smartcore provides to the users, let's try the new `RealNumber` trait:

In [7]:
use smartcore::numbers::realnum::*;  // this line is not needed, just to be explicit

println!("{:?}", 1.0.sigmoid());     // 0.7310585786300049
println!("{:?}", 41.0.sigmoid());     // 1.
println!("{:?}", (-41.0).sigmoid());  // 0


0.7310585786300049
1.0
0.0


As you can see `RealNumber` extends the usual `float` with some convenience methods.

All the `numbers` traits implement some of Rust standard library's primitives with usefull methods, like `sigmoid`.

Smartcore `numbers` provides traits that are composition of [`num-traits` library's](https://docs.rs/num-traits/latest/num_traits/) base traits:
* `basenum::Number`: . The most relavant traits are `Bounded` and `Copy` but you can look up all the ones that are part of the set.
* `floatnum::FloatNumber`: a trait that is the composition of `Number + Float + Signed`
* `realnum::RealNumber`: a trait to specify Reals (Rational and Irrational), it is the composition of `Number + Float`

This is absolutely transparent to the user. Whatever Rust primitive (strings will also be implemented soon enough) you decide to use in your matrices the traits system will check that the constraints are satisfied, making everything safer and providing compile-time goodies. 

The traits system is not limited to these foundamental traits, you will see more theoretically-defined traits applied to matrices like the ones in `smartcore::linalg::traits` that allow properties to be attached to 2D arrays with particular characteristics. 


## -> where to start

*If you want to just jump into running algorithms on your data*, the best place to start is instantiating a `smartcore::linalg::basic::matrix::DenseMatrix`; with this handy structure you can leverage all the functionalities provided by the traits system with a simple interface. For the usage of  `DenseMatrix` see the other notebooks where procedures are presented.

*If you are interested in understanding how Smartcore works* because you are thinking about contributing or you are just curious, keep reading here and you will find some nuggets.

## vectors (1D arrays)

Smartcore also provides useful extensions for Rust's `Vec`, you can find those in `src/linalg/basic/vector`. For example this implementation:
```
impl<T: Debug + Display + Copy + Sized> Array1<T> for Vec<T> { ... }
```
extends a generic `Vec<T>` with Smartcore's custom capabilities from `smartcore::lianalg::basic::arrays::Array1`, like creting a mutable instance from a `Range` (slice):
```
fn slice_mut<'b>(&'b mut self, range: Range<usize>) -> Box<dyn MutArrayView1<T> + 'b>
```

Other interesting convenience methods provided by this module:
* `from_iterator`
* `iterator_mut`
* `from_vec_slice`
* ...

Linear Algebra operations are collected in:
```
pub trait MutArray<T: Debug + Display + Copy + Sized, S>: Array<T, S> { ... }
```

Let's see some examples leveraging mutability:


In [17]:
// getting an element

// we  need this trait in scope to handle mutable arrays
use smartcore::linalg::basic::arrays::MutArray;

// this is a mutable Vec<i32>
let mut x = vec![1, 2, 3];

// this unwraps the Option<&T> and prints the value
println!("original value: {:?}", *x.get(2).unwrap());

// this sets the value at position 1
x.set(1, 101);

println!("set value: {:?}", x.get(1).unwrap());

original value: 3
set value: 101


In [49]:
// we  need this trait in scope to use iterator
use smartcore::linalg::basic::arrays::Array;

// iterate over one of the axis of the array
// use `Array::iterator` on a Vec<i32>
let v: Vec<i32> = vec![1, 2, 3];
let mut pow_: Vec<i32> = vec![];
v.iterator(0).for_each(|v| pow_.push(v.pow(2)));
println!("initial vector: {:?}", v);
println!("power of 2 vector: {:?}", pow_);

// same but with mutability
// use `Array::iterator_mut` on a Vec<i32>
vec![1, 2, 3].iterator_mut(0)   // instantiate a mutable iterator on axis 0
    .for_each(|v| {
        *v = -*v;                         // change sign to all the elements
        println!("{:?}", v.clone());
    }
);                


// use of axis
// this line panics as an `Array` has no second axis
// is this the right behaviour? should we return a `Result` instead?
let _ = vec![1, 2, 3].iterator(1);

initial vector: [1, 2, 3]
power of 2 vector: [1, 4, 9]
-1
-2
-3


thread '<unnamed>' panicked at 'For one dimensional array `axis` should == 0', /home/lorenzo/.cargo/git/checkouts/smartcore-91f9710bf445643e/42d017e/src/linalg/basic/vector.rs:32:9
stack backtrace:
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
